<a href="https://colab.research.google.com/github/srinijammula/Web_Scraping/blob/main/Website_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install zenrows

In [ ]:
#to bypass cloudflare
from zenrows import ZenRowsClient

client = ZenRowsClient("bfee92b72f345a9bf3e4005b5da9e3244bc8b75d")
url = "https://journals.sagepub.com/toc/JMX/current"
params = {"js_render":"true"}

response = client.get(url, params=params)
print(response)

<Response [200]>


In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
#class to extract all journals content
tables=soup.find(class_='table-of-content')

In [ ]:
#extract links of each journal
article_links = [a['href'] for a in tables.select('a[data-id="toc-article-title"]')]

print("Extracted Article Links:")
for link in article_links:
    print(link)

Extracted Article Links:
/doi/abs/10.1177/00222429231224748
/doi/abs/10.1177/00222429241236142
/doi/abs/10.1177/00222429231224333
/doi/abs/10.1177/00222429241227145
/doi/abs/10.1177/00222429241231236
/doi/abs/10.1177/00222429231223420
/doi/abs/10.1177/00222429241239892
/doi/abs/10.1177/00222429231222269
/doi/abs/10.1177/00222429241270362


In [ ]:
#create a dictionary to store information of each journal
d={'Title':[],'Authors':[],'First published date':[],'Date of Issue':[],'Abstract':[]}

In [ ]:
#function to extract details by article url
def fetch_article_details(article_url):
    base_url = "https://journals.sagepub.com"
    full_url = base_url + article_url

    #extract each article content
    article_response = client.get(full_url, params=params)
    article_soup = BeautifulSoup(article_response.content, 'html.parser')

    #title
    article_title = article_soup.find('h1',property='name').text.strip()
    d['Title'].append(article_title)
    print(article_title)

    #authors
    authors_section = article_soup.find(class_='core-authors')
    if authors_section:
      authors_list=[]
      authors = authors_section.find_all(property='author')
      for author in authors:
          given_name = author.find(property='givenName').text.strip()
          family_name = author.find(property='familyName').text.strip()
          full_name=given_name+family_name
          authors_list.append(full_name)
          print(full_name)
      d['Authors'].append(', '.join(authors_list))
    else:
      d['Authors'].append('')

    #first issue date and issue date
    first_published = article_soup.find_all('b', class_='core-label')[0].next_sibling.strip()[2:]
    issue_published = article_soup.find_all('b', class_='core-label')[1].next_sibling.strip()[2:]
    d['First published date'].append(first_published)
    d['Date of Issue'].append(issue_published)
    print("First Published Date:", first_published)
    print("Issue Published Date:", issue_published)

    #abstract
    abstract_section = article_soup.find(property='abstract')
    if abstract_section:
      abstract = abstract_section.find(role='paragraph').text.strip()
      d['Abstract'].append(abstract)
      print("Abstract:", abstract)
    else:
      d['Abstract'].append('')

#call fetch function for all links
for link in article_links:
  fetch_article_details(link)
  print('-------------------------------------------')

The Caring Machine: Feeling AI for Customer Care
Ming-HuiHuang
Roland T.Rust
First Published Date: December 21, 2023
Issue Published Date: September 2024
Abstract: Customer care is important for its role in relationship building. This role has traditionally been performed by human customer agents; however, the emergence of interactive generative AI (GenAI) shows potential for using AI for customer care in emotionally charged interactions. Bridging practice and the academic literatures in marketing and computer science, this article develops an AI-enabled customer care journey, from accurate emotion recognition to empathetic response, emotional management support, and, finally, the establishment of an emotional connection. Marketing requirements for each of the stages are derived from in-depth interviews with top managers and a survey of chief marketing officers. By juxtaposing these requirements against the current feeling capabilities of GenAI, the authors highlight the technological 

In [ ]:
#convert dictionary to dataframe
import pandas as pd

df=pd.DataFrame(d)
df

,Title,Authors,First published date,Date of Issue,Abstract
0,The Caring Machine: Feeling AI for Customer Care,"Ming-HuiHuang, Roland T.Rust","December 21, 2023",September 2024,Customer care is important for its role in rel...
1,Consequences of Marketing Asset Accountability...,"PeterGuenther, MiriamGuenther, Bryan A.Lukas, ...","February 15, 2024",September 2024,Marketing scholars have extensively studied ma...
2,Developing Strengths or Remedying Weaknesses? ...,"QihuiChen, YajinWang, YingZhang","December 20, 2023",September 2024,Does parents’ status motivation affect their e...
3,More Likely to Pay but Less Engaged: The Effec...,"JoyLu, Eric T.Bradlow, J. WesleyHutchinson","January 8, 2024",September 2024,Following trends in entertainment streaming se...
4,"So, Sue Me…If You Can! How Legal Changes Dimin...","Arvid O.I.Hoffmann, Chee S.Cheong, Hoàng-LongP...","February 1, 2024",September 2024,"Research examining the antecedents of recalls,..."
5,Targeting Nearby Influencers: The Acceleration...,"JacobGoldenberg, AndreasLanz, DanielShapira, F...","December 14, 2023",September 2024,"On user-generated content platforms, individua..."
6,How Own Delivery Services Influence Customer B...,"BanggangWu, YuboChen, Prasad A.Naik","March 4, 2024",September 2024,Online retailers worldwide invest beyond their...
7,Collaborating to Innovate: Balancing Strategy ...,"NehalElhelaly, SouravRay","December 11, 2023",September 2024,"When a firm collaborates with its suppliers, i..."
8,Corrigendum to “The Caring Machine: Feeling AI...,,"August 8, 2024",September 2024,


In [ ]:
#convert dataframe to csv
df.to_csv('journals.csv',index=False)